## 데이터 분할하는 폴더 만들기

In [1]:
import os
import shutil

original_dataset_dir = './torch_project/dataset'
classes_list = os.listdir(original_dataset_dir) # 디렉토리 모든 폴더목록을 가져옴. 즉 모든 class들 가져옴.
base_dir = './splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))

In [2]:
import math
for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss) # 클래스별 원본 경로
    fnames = os.listdir(path)
    
    train_size = math.floor(len(fnames)*0.6)
    validation_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)
    
    train_fnames = fnames[:train_size]
    print('Train_size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname) # 이미지가 존재하는 경로
        dst = os.path.join(os.path.join(train_dir, clss), fname) # 학습용 데이터셋으로 나눈 경로
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:(validation_size+train_size)]
    print('Validation_size(',clss,'): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname) # 이미지가 존재하는 경로
        dst = os.path.join(os.path.join(validation_dir, clss), fname) # 학습용 데이터셋으로 나눈 경로
        shutil.copyfile(src, dst)
    
    test_fnames = fnames[(validation_size+train_size):(test_size + validation_size + train_size)]
    print('Test_size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname) # 이미지가 존재하는 경로
        dst = os.path.join(os.path.join(test_dir, clss), fname) # 학습용 데이터셋으로 나눈 경로
        shutil.copyfile(src, dst)

Train_size( Strawberry___healthy ):  273
Validation_size( Strawberry___healthy ):  91
Test_size( Strawberry___healthy ):  91
Train_size( Grape___Black_rot ):  708
Validation_size( Grape___Black_rot ):  236
Test_size( Grape___Black_rot ):  236
Train_size( Potato___Early_blight ):  600
Validation_size( Potato___Early_blight ):  200
Test_size( Potato___Early_blight ):  200
Train_size( Cherry___Powdery_mildew ):  631
Validation_size( Cherry___Powdery_mildew ):  210
Test_size( Cherry___Powdery_mildew ):  210
Train_size( Tomato___Target_Spot ):  842
Validation_size( Tomato___Target_Spot ):  280
Test_size( Tomato___Target_Spot ):  280
Train_size( Peach___healthy ):  216
Validation_size( Peach___healthy ):  72
Test_size( Peach___healthy ):  72
Train_size( Potato___Late_blight ):  600
Validation_size( Potato___Late_blight ):  200
Test_size( Potato___Late_blight ):  200
Train_size( Tomato___Late_blight ):  1145
Validation_size( Tomato___Late_blight ):  381
Test_size( Tomato___Late_blight ):  381

## 학습을 위한 모델 준비

In [3]:
import torch
use_mps = torch.backends.mps.is_available() # m1 gpu 가속
device = torch.device('mps' if use_mps else 'cpu')
print(device)

batch_size = 256
epoch = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)), # 이미지 크기를 64x64로 조정
                                    transforms.ToTensor()])# 이미지에 대해 정규화 진행
train_dataset = ImageFolder(root='./splitted/train/', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val/', transform=transform_base)

# ImageFolder 메서드는 데이터셋을 불러오는데 활용. 하나의 클래스가 하나의 폴더에 대응되는 계층적 데이터셋에 좋음. root를 통해 경로, transform을 통해 전처리를 진행할 방법 지정.

from torch.utils.data import DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4) # 이미지 데이터를 mini batch로 조정.
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

mps


## 베이스라인 모델 설계

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # 입력 채널 수, 출력 채널 수, 커널크기
        self.pool = nn.MaxPool2d(2, 2) # 커널크기, stride
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(4096, 512) # 64*64 = 4096으로 flatten
        self.fc2 = nn.Linear(512, 33)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training) # 25%의 node를 dropout
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
model_base = Net().to(device) # 장비 할당
optimizer = optim.Adam(model_base.parameters(), lr=0.001) # optimizer에 parameter를 설정

In [5]:
def train(model, train_loader, optimizer):
    model.train() # 학습모드
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 이전 epoch의 미분값이 저장되어있는 것을 초기화
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward() # 역전파 진행
        optimizer.step()
        
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad(): # parameter를 업데이트 하지 않도록 설정
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim = True)[1] # 가장 높은 확률값의 인덱스를 예측값으로 지정
            correct += pred.eq(target.view_as(pred)).sum().item() # target의 텐서 구조를 pred 텐서 구조와 같은 모양으로 정렬. 만약 같을 경우 eq()가 1을 반환.
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [6]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        time_elapsed = time.time() - since
        print('---------- epoch {} ----------'.format(epoch))
        print('train loss : {:.4f}, accuracy : {:.2f}%'.format(train_loss, train_acc))
        print('val loss : {:.4f}, accuracy : {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    model.load_state_dict(best_model_wts)
    return model

In [7]:
base = train_baseline(model_base, train_loader, val_loader, optimizer, epoch)

---------- epoch 1 ----------
train loss : 1.5108, accuracy : 56.20%
val loss : 1.5119, accuracy : 56.18%
Completed in 1m 35.313821s
---------- epoch 2 ----------
train loss : 1.0421, accuracy : 68.12%
val loss : 1.0677, accuracy : 67.32%
Completed in 1m 27.277847s
---------- epoch 3 ----------
train loss : 0.6458, accuracy : 79.50%
val loss : 0.6936, accuracy : 77.92%
Completed in 1m 32.276491s
---------- epoch 4 ----------
train loss : 0.4937, accuracy : 84.43%
val loss : 0.5593, accuracy : 82.76%
Completed in 1m 33.244026s
---------- epoch 5 ----------
train loss : 0.3897, accuracy : 87.62%
val loss : 0.4719, accuracy : 85.05%
Completed in 1m 35.541857s
---------- epoch 6 ----------
train loss : 0.3033, accuracy : 90.90%
val loss : 0.3975, accuracy : 87.65%
Completed in 1m 44.082572s
---------- epoch 7 ----------
train loss : 0.2573, accuracy : 92.09%
val loss : 0.3660, accuracy : 88.61%
Completed in 1m 44.286130s
---------- epoch 8 ----------
train loss : 0.2334, accuracy : 92.88%


In [8]:
torch.save(base, 'baseline.pt')

## Transfer Learning

In [13]:
data_transforms = {
    'train' : transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),# 이미지 무작위 좌우반전
        transforms.RandomVerticalFlip(), # 이미지 무작위 상하반전
        transforms.RandomCrop(52),# 이미지 무작위로 52x52로 변경
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = './splitted'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), 
                                transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

In [14]:
from torchvision import models

resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 33) # 모델 마지막 layer수를 조정
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr = 0.001) # requires_grad = True 인 layer들에만 학습 진행
from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) # 7epochs 마다 0.1을 곱하여 학습율 변경

In [15]:
ct = 0
for child in resnet.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False
            
# 예를들어 layer가 10개가 있으면 0~5는 갱신이 안되고, 6~10은 갱신됨.

In [23]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs = 25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('----------epoch {}----------'.format(epoch + 1))
        since = time.time()
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # phase == 'train'일때만 가중치 갱신되도록 설정
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate:', l_r)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict_dict())
        
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    print('Best val acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model

In [24]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=epoch)
torch.save(model_resnet50, 'resnet50.pt')

----------epoch 1----------
learning rate: [0.001]
train Loss: 3.2896 acc: 0.0000
val Loss: 3.3293 acc: 0.0000
Completed in 5m 16s
----------epoch 2----------


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x13e925dc0>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1407, in __del__
    self._shutdown_workers()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1371, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/multiprocessing/connection.py", line 936, in wait
    ready = selector.select(timeout)
  File 

KeyboardInterrupt: 